# Web Scraping

In [ ]:
import pandas as pd
#import numpy as np
import requests
from bs4 import BeautifulSoup
import datetime
import json
import re

In [ ]:
data=pd.read_csv('sub_isbn.csv')

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
result={}
for isbn in range(0,50000):
    print(isbn)
    ISBN=data.values[isbn][0].zfill(10)
    amazon_url  = 'http://www.amazon.com/dp/'+ISBN 
    response = requests.get(amazon_url,headers = headers)
    if response.status_code == 200: 
        page_response = BeautifulSoup(response.content,'lxml') 
        
        try:
            book_name = page_response.find('span', id = 'productTitle').get_text()
            try:
                author = page_response.find('div',id='byline').find('span',class_='author notFaded').find('a',{"class":'a-link-normal contributorNameID'}).get_text()
            except:
                try:
                    author = page_response.find('div',id='byline').find('span',class_='author notFaded').find('a',{"class":'a-link-normal'}).get_text()
                except:
                    info={'Name':None,'author':None,'pages':None,'publisher':None,'language':None,'rank':None,'category':None,'tags':None} 
            details = page_response.find('table',id='productDetailsTable').find_all('li')
            for row in details:
                if 'page' in row.get_text().lower():
                    pages=row.get_text().split(':')[1][1:]
                if 'publisher:' in row.get_text().lower():
                    publisher=row.get_text().split(':')[1][1:]
                if 'language:' in row.get_text().lower():
                    language=row.get_text().split(':')[1][1:]
                if 'sellers rank:' in row.get_text().lower():
                    rank=row.get_text()
                    totalrank=rank[rank.index('#')+1:rank.index('(')-1].split(' ')[0]
                    sr = row.find_all('li', class_='zg_hrsr_item')
                    c=re.sub("[^a-zA-Z0-9&>,'-]", ' ', sr[0].get_text())
                    category=c.split('>')[1].strip()
                    tag = []
                    for i in range(len(sr)):
                        sub = re.sub("[^a-zA-Z0-9&>,'-]",
                                     ' ', sr[i].get_text())
                        sub = re.sub(' in ', '', sub)
                        sub = re.sub('Books', '', sub)
                        sub = sub.split('>')
                        for j in range(len(sub)):
                            sub[j] = sub[j].strip()
                        tag.append(sub[-1])

                
            reviews_list_1 = []
            reviews_list_2 = []
            a = page_response.find(
                'div', {'id': 'most-recent-reviews-content'})
            b = a.find_all('div', {'data-hook': 'recent-review'})
            for i in b:
                c = i.find('span', {'class': 'a-icon-alt'}).get_text()
                reviews_list_1.append(c)
                x = list(range(len(reviews_list_1)))
                y = dict(zip(x, reviews_list_1))
            for i in b:
                d = i.find(
                    'span', {'data-hook': 'review-body-recent'}).get_text()
                reviews_list_2.append(d)
                w = list(range(len(reviews_list_2)))
                z = dict(zip(w, reviews_list_2))
                
    
            for t in page_response.find_all('span', {'class': 'a-size-medium a-color-price offer-price a-text-normal'}):
                price_ = float(t.get_text()[1:])
                
            ########################### star distrution and average score               
            ratings_dict = {}
            for i in range (1,6):
                rating_key=i
                try:
                    rating_value=page_response.find('a',class_="a-size-base a-link-normal "+str(i)+"star histogram-review-count").get_text()
                except AttributeError:
                    rating_value='0%'
                ratings_dict.update({rating_key: rating_value})
            rating_key='average score'
            average_score=0
            for i in range(1,6):
                if type(ratings_dict[i])!=int:
                    a=round((i*float(ratings_dict[i].strip('%'))/100),2)
                    average_score+=a
            ratings_dict.update({rating_key: average_score})
            ########################### customer reviews
            try:
                reviews_value=page_response.find('span',id="acrCustomerReviewText").get_text()
            except AttributeError:
                reviews_value='0 reviews'
            ###########################   introduction
            try:
                intro=str(page_response.find_all('noscript')[1])
                intro=re.compile(r'<[^>]+>').sub('',intro)
                intro = re.sub('[^a-zA-Z0-9-_*.,]', ' ', intro)
            except AttributeError:
                intro=None
            info={'Name':book_name,'price': price_, 'author':author,'pages':pages,'publisher':publisher,'language':language,
                  'rank':totalrank,'category':category,'tags':tag,'Reviews': y,'Stars': z,'Ratings':ratings_dict,'Cutomers Reviews':reviews_value,
                  'Introduction':intro}                
        except (AttributeError, ConnectionError, IndexError):
            info={'Name':None,'price':None, 'author':None,'pages':None,'publisher':None,'language':None,
                  'rank':None,'category':None,'tags':None,'Reviews': None,'Stars': None,'Ratings':None,'Cutomers Reviews':None,
                  'Introduction':None} 
    result[ISBN]=info

In [ ]:
result

In [ ]:
import json
with open('Output.json', 'w') as fp:
    json.dump(result, fp)